### hajimeruzou!!!

In [1]:
import cupy
import numpy as np
import math
import time
import numba
from numba import cuda
from numba import njit
from numba import prange

cupy.cuda.set_allocator(None)

date = "6/24"
N_PATHS = 50*10000 # took off a 0
# N_PATHS = 2 # took off a 0
N_STEPS = 365
T = 365   # was 1, is prolly useless now as im using N_STEPS instead

S0 = 10.0
Ko = 1*S0      #knock in, next is knock out
Ki = 0.8*S0
sigma = 0.35
mu = 0.025
r = 0.025
MONTHS = [0, 31,59,90,120,151,181,212,243, 273,304,334]
MONTHS = [0, 31,59,90,120,153,181,212,245, 273,304,335]        #modified dates
pot = 20      # potential investment multiplier (percent)



In [2]:
split = date.split("/")
temp_months = MONTHS.copy()

startday = int(split[1])

for month in range(len(MONTHS)):
    temp_months[month] += startday       # adds the amont of days passed to each month

for month in range(int(split[0])):
    temp_months[month] += 365


print(startday)
startday += MONTHS[int(split[0])-1] +1

print (temp_months)
temp_months = cupy.asnumpy(temp_months)     # is very important for speedup!!!
print(startday)
print (temp_months)

24
[389, 420, 448, 479, 509, 542, 205, 236, 269, 297, 328, 359]
178
[389 420 448 479 509 542 205 236 269 297 328 359]


In [3]:
# if True:
#     temp_months = MONTHS
#     startday = 0

In [4]:
@njit(fastmath=True)        #fastmath does jack shit

def brownian_motion(sp, mu, sigma, T):      # sp= stock price
    b_motion = np.random.randn()
    dt = 1/T
    ds = sp * mu * dt + sp * sigma * b_motion * math.sqrt(dt)  
    return ds
    

In [5]:
@njit(fastmath=True, parallel=True)        #fastmath does jack shit

def snowball_price(s_0, mu, sigma, T, Ki, Ko, temp_months, pot,r):
    min = s_0
    snowball_path = [s_0]     # array of size 1
    ko = False
    ki = False
    for t in prange(T):
        # path
        ds = brownian_motion(snowball_path[t], mu, sigma, T)
        snowball_path.append(snowball_path[t]+ds)
        # pricing and break if need be
        if snowball_path[t+1] <= Ki:
            ki = True
            if min > snowball_path[t+1]:
                min = snowball_path[t+1]        # getting lowest drop
        
        if t+1+startday in temp_months:     #startday used to fake a start date in code
            if snowball_path[t+1] >= Ko:
                price =  pot * t/365     # should turn t into int
                # return snowball_path, price
                return price * math.exp(-r * t/T)   # accounting for r

            # if snowball_path[t+1] <= Ki:
            #     price = 0
            #     return price * math.exp(-r * t/T)
    
    # did not get knocked up or down
    price = pot
    if ki and snowball_path[-1] <= s_0:          # blo got knocked down and never recovered
        price = snowball_path[-1] - s_0
    elif ki:          # blo got knocked down for a bit but finished above Ki
        price =0
    return price * math.exp(-r)



In [6]:
@njit(fastmath=True, parallel=True)        #fastmath does jack shit

def snowball_price_faked(s_0, mu, sigma, T, Ki, Ko, temp_months, pot,r, shiftp, blipp, regp, shiftdate, blipdate):
    min = s_0
    snowball_path = [s_0]     # array of size 1
    ko = False
    ki = False
    for t in range(T):
        # path
        # ds = brownian_motion(snowball_path[t], mu, sigma, T)




            # FAKED PART HERE
        # if t >= 300 :
        #     snowball_path.append(11)
        # elif t ==75: 
        #     snowball_path.append(9)
        # else: snowball_path.append(9)

            # FAKED PART HERE
        if t >= shiftdate :
            snowball_path.append(shiftp)
        elif t ==blipdate: 
            snowball_path.append(blipp)
        else: snowball_path.append(regp)





        # pricing and break if need be
        if snowball_path[t+1] <= Ki:
            ki = True
            if min > snowball_path[t+1]:
                min = snowball_path[t+1]        # getting lowest drop
        
        if t+1+startday in temp_months:     #startday used to fake a start date in code
            if snowball_path[t+1] >= Ko:
                price =  pot * t/365     # should turn t into int
                # return snowball_path, price
                return price * math.exp(-r * t/T)   # accounting for r

            # if snowball_path[t+1] <= Ki:
            #     price = 0
            #     return price * math.exp(-r * t/T)
    
    # did not get knocked up or down
    price = pot
    if ki and snowball_path[T] <= s_0:          # blo got knocked down and never recovered
        price = snowball_path[T] - s_0
    elif ki:          # blo got knocked down for a bit but finished above Ki
        price =0
    return price * math.exp(-r)



### Runin da CPU

In [7]:
shiftdate = 112
blipdate = 75

highp = 11
medp = 9
lowp = 7

In [8]:
njit( fastmath = True, parallel=True) # parallel also does jack shit
def monte_carlo_andtheholygrail_cpu(d_s, s_0, mu, sigma, T, Ki, Ko, temp_months, pot,r):
    for n in prange(N_PATHS):           # prange is parallel range
        # returns = snowball_price(s_0, mu, sigma, T, Ki, Ko, temp_months, pot,r)

        #       premature exit
        returns = snowball_price(s_0, mu, sigma, T, Ki, Ko, temp_months, pot,r)
                                                                                    # shiftp,  blipp,  regp, 

        d_s[n] = returns


njit( fastmath = True, parallel=True) # parallel also does jack shit
def monte_carlo_andtheholygrail_cpu_faked(d_s, s_0, mu, sigma, T, Ki, Ko, temp_months, pot,r, shiftp, blipp, regp, shiftdate, blipdate):
    for n in prange(N_PATHS):           # prange is parallel range
        # returns = snowball_price(s_0, mu, sigma, T, Ki, Ko, temp_months, pot,r)

        #       premature exit
        returns = snowball_price_faked(s_0, mu, sigma, T, Ki, Ko, temp_months, pot,r, shiftp ,  blipp,   regp, shiftdate, blipdate)
                                                                                    # shiftp,  blipp,  regp, 

        d_s[n] = returns

In [9]:
njit( fastmath = True, parallel=True) # parallel also does jack shit
output =  np.zeros(N_PATHS, dtype=np.float32)
output1 =  np.zeros(N_PATHS, dtype=np.float32)
output2 =  np.zeros(N_PATHS, dtype=np.float32)
output3 =  np.zeros(N_PATHS, dtype=np.float32)
output4 =  np.zeros(N_PATHS, dtype=np.float32)
output5 =  np.zeros(N_PATHS, dtype=np.float32)

s = time.time()


monte_carlo_andtheholygrail_cpu(output, np.float32(S0), np.float32(mu), 
                            np.float32(sigma), np.int32(T),np.float32(Ki),
                            np.float32(Ko), temp_months, np.float32(pot),
                             np.float32(r))
v = output.mean()
e = time.time()
print('time', e-s, 'v', v, 'avg time', (e-s)/N_PATHS)




# #                    premature exit
# monte_carlo_andtheholygrail_cpu_faked(output1, np.float32(S0), np.float32(mu), 
#                             np.float32(sigma), np.int32(T),np.float32(Ki),
#                             np.float32(Ko), temp_months, np.float32(pot),
#                              np.float32(r), highp ,  medp,   medp, shiftdate, blipdate)
#                                          # shiftp,  blipp,  regp, 
# v = output1.mean()
# e = time.time()
# print('v', v)
# #                    no exit
# monte_carlo_andtheholygrail_cpu_faked(output2, np.float32(S0), np.float32(mu), 
#                             np.float32(sigma), np.int32(T),np.float32(Ki),
#                             np.float32(Ko), temp_months, np.float32(pot),
#                              np.float32(r),medp ,  medp,   medp, shiftdate, blipdate)
#                                          # shiftp,  blipp,  regp, 
# v = output2.mean()
# e = time.time()
# print('v', v)
# #                    dip & recover
# monte_carlo_andtheholygrail_cpu_faked(output3, np.float32(S0), np.float32(mu), 
#                             np.float32(sigma), np.int32(T),np.float32(Ki),
#                             np.float32(Ko), temp_months, np.float32(pot),
#                              np.float32(r), medp ,  lowp,   medp, shiftdate, blipdate)
#                                          # shiftp,  blipp,  regp, 
# v = output3.mean()
# e = time.time()
# print('v', v)
# #                    dip & nah id lose
# monte_carlo_andtheholygrail_cpu_faked(output4, np.float32(S0), np.float32(mu), 
#                             np.float32(sigma), np.int32(T),np.float32(Ki),
#                             np.float32(Ko), temp_months, np.float32(pot),
#                              np.float32(r), lowp ,  lowp,   medp, shiftdate, blipdate)
#                                          # shiftp,  blipp,  regp, 
# v = output4.mean()
# e = time.time()
# print('v', v)
# #                    dip & nah id win
# monte_carlo_andtheholygrail_cpu_faked(output5, np.float32(S0), np.float32(mu), 
#                             np.float32(sigma), np.int32(T),np.float32(Ki),
#                             np.float32(Ko), temp_months, np.float32(pot),
#                              np.float32(r), highp ,  lowp,   medp, shiftdate, blipdate)
#                                          # shiftp,  blipp,  regp, 
# v = output5.mean()
# e = time.time()
# print('v', v)







c:\Users\xyzqadmin\anaconda3\Lib\site-packages\numba\parfors\parfor.py:2395: NumbaPerformanceWarning: 
prange or pndindex loop will not be executed in parallel due to there being more than one entry to or exit from the loop (e.g., an assertion).

File "..\..\AppData\Local\Temp\ipykernel_32028\1868880379.py", line 8:
<source missing, REPL/exec in use?>

  warnings.warn(
c:\Users\xyzqadmin\anaconda3\Lib\site-packages\numba\core\typed_passes.py:336: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "..\..\AppData\Local\Temp\ipykernel_32028\1868880379.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


time 2.6047768592834473 v 2.4351046 avg time 5.209553718566895e-06


### Running the GPUs
this first box is depreciated, was used to throw all the code into a single method to prevent a fuck up that no longer exists. All that is left is some rly scrunched up code.

In [22]:
randoms_gpu = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)

In [32]:
@cuda.jit               # defualt GPU
def monte_carlo_andtheholygrail_gpu(d_s, s_0, mu, sigma, T, Ki, Ko, temp_months, 
                                    pot,r,d_normals,  snowball_path, snowball_path_holder):
    

    # for shared memory (non)optimization
    shared = cuda.shared.array(shape=0, dtype=numba.float32)
    # load to shared memory
    path_offset = cuda.blockIdx.x * cuda.blockDim.x

    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x

    for n in range(ii, N_PATHS, stride):
        min = s_0

            #cant instantiate lists in here ffs, need to pass it in instead
        snowball_path_holder[n][0] = s_0

        earlyexit = False
        ki = False
        mald = False
        for t in range(T):
            # pre shared memory b_motion
            # b_motion = d_normals[n + t * N_PATHS]

            # post shared memory b_motion
            shared[cuda.threadIdx.x] = d_normals[path_offset + cuda.threadIdx.x + t * N_PATHS]

            dt = 1/T
            # pre shared memory b_motion
            # ds = snowball_path[t] * mu * dt + snowball_path[t] * sigma * b_motion * math.sqrt(dt) 
            # post shared memory b_motion
            ds = snowball_path_holder[n][t] * mu * dt + snowball_path_holder[n][t] * sigma * shared[cuda.threadIdx.x] * math.sqrt(dt) 
                    # no adjusting list sizes in cuda :(
            # snowball_path.append(snowball_path[t]+ds)
            snowball_path_holder[n][t+1] = snowball_path_holder[n][t] + ds
            

            # ki = snowball_path[t] + ds
            if snowball_path_holder[n][t+1] <= Ki:
                ki = True
                if min > snowball_path_holder[n][t+1]:
                    min = snowball_path_holder[n][t+1]        # getting lowest drop

            # if t+1+startday in temp_months:     #startday used to fake a start date in code
            #     if snowball_path[t+1] >= Ko:
            #         price =  pot * t/365     # should turn t into int
            #         # return snowball_path, price
            #         d_s[n] =  price * math.exp(-r * t/T)   # accounting for r
            #         earlyexit = True
            #         break


            if not mald:
                for month in (0,1,2,3,4,5,6,7,8,9,10,11):                # need to do this instead because contains (in) and range are disabled
                    if t+1+startday == temp_months[month]:     #startday used to fake a start date in code
                        # price = t+1+startday
                        if snowball_path_holder[n][t+1] >= Ko:
                            price =  pot * t/365     # should turn t into int
                            # return snowball_path, price
                            d_s[n] =  price * math.exp(-r * t/T)   # accounting for r
                            snowball_path_holder[n][-1] = d_s[n]            
                            earlyexit = True
                            mald = True
                            # print("blo got fucked\n")
                            break

        
        if not earlyexit:       # to prevent early exit getting out of bdds error
            # did not get knocked up or down
            price = pot
            # t  =T 
                        # CAN'T USE T CUZ CUDA IS FUCKING SHIT so use -1 instead
                        # or not ig T works now :sob:
            if ki and snowball_path_holder[n][T] <= s_0:          # blo got knocked down and never recovered
                # price = min - s_0
                price = snowball_path_holder[n][T] - s_0
            elif ki and snowball_path_holder[n][T] <= Ko:          # blo got knocked down for a bit but finished above Ki
                price =0
            d_s[n] = price * math.exp(-r)
            snowball_path_holder[n][-1] = d_s[n]    


In [24]:
# @cuda.jit         # gpu code run w/ cpu
# @njit(fastmath=True, parallel=True)
@njit(fastmath=True)
def monte_carlo_andtheholygrail_gpu_but_cpu(d_s, s_0, mu, sigma, T, Ki, Ko, temp_months, 
                                    pot,r,d_normals,  snowball_path, snowball_path_holder_but_cpu):
    

    # # for shared memory (non)optimization
    # shared = cuda.shared.array(shape=0, dtype=numba.float32)
    # # load to shared memory
    # path_offset = cuda.blockIdx.x * cuda.blockDim.x

    # ii - overall thread index
    # ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    # stride = cuda.gridDim.x * cuda.blockDim.x

    # for n in range(ii, N_PATHS, stride):
    for n in range(N_PATHS):
        min = s_0

            #cant instantiate lists in here ffs, need to pass it in instead     snowball_path_holder_but_cpu[n]
        # snowball_path[0] = s_0
        snowball_path_holder_but_cpu[n][0] = s_0

        earlyexit = False
        ki = False
        mald = False
        for t in range(T):
            # pre shared memory b_motion
            b_motion = d_normals[n + t * N_PATHS]

            # # post shared memory b_motion
            # shared[cuda.threadIdx.x] = d_normals[path_offset + cuda.threadIdx.x + t * N_PATHS]

            dt = 1/T
            # pre shared memory b_motion
            # ds = snowball_path[t] * mu * dt + snowball_path[t] * sigma * b_motion * math.sqrt(dt) 
            ds = snowball_path_holder_but_cpu[n][t] * mu * dt + snowball_path_holder_but_cpu[n][t] * sigma * b_motion * math.sqrt(dt) 
            # post shared memory b_motion
            # ds = snowball_path[t] * mu * dt + snowball_path[t] * sigma * shared[cuda.threadIdx.x] * math.sqrt(dt) 
                    # no adjusting list sizes in cuda :(
            # snowball_path.append(snowball_path[t]+ds)
            # snowball_path[t+1] = snowball_path[t] + ds
            snowball_path_holder_but_cpu[n][t+1] = snowball_path_holder_but_cpu[n][t] + ds
            

            # if snowball_path[t+1] <= Ki:
            if snowball_path_holder_but_cpu[n][t+1] <= Ki:
                ki = True
                if min > snowball_path_holder_but_cpu[n][t+1]:
                    min = snowball_path_holder_but_cpu[n][t+1]        # getting lowest drop

            # if t+1+startday in temp_months:     #startday used to fake a start date in code
            #     if snowball_path[t+1] >= Ko:
            #         price =  pot * t/365     # should turn t into int
            #         # return snowball_path, price
            #         d_s[n] =  price * math.exp(-r * t/T)   # accounting for r
            #         earlyexit = True
            #         break


            if not mald:
                for month in (0,1,2,3,4,5,6,7,8,9,10,11):                # need to do this instead because contains (in) and range are disabled
                    if t+1+startday == temp_months[month]:     #startday used to fake a start date in code
                        # price = t+1+startday
                        if snowball_path_holder_but_cpu[n][t+1] >= Ko:
                            price =  pot * t/365     # should turn t into int
                            # return snowball_path, price
                            d_s[n] =  price * math.exp(-r * t/T)   # accounting for r
                            snowball_path_holder_but_cpu[n][-1] = d_s[n]                                                                ## added line for checking payoff
                            earlyexit = True
                            mald = True
                            # print("blo got fucked\n")
                            break

        
        if not earlyexit:       # to prevent early exit getting out of bdds error
            # did not get knocked up or down
            price = pot
            # t  =T 
                        # CAN'T USE T CUZ CUDA IS FUCKING SHIT so use -1 instead
                        # or not ig T works now :sob:
            if ki and snowball_path_holder_but_cpu[n][T] <= s_0:          # blo got knocked down and never recovered
                # price = min - s_0
                price = snowball_path_holder_but_cpu[n][T] - s_0
            elif ki and snowball_path_holder_but_cpu[n][T] <= Ko:          # blo got knocked down for a bit but finished above Ki
                price =0
            d_s[n] = price * math.exp(-r)
            snowball_path_holder_but_cpu[n][-1] = d_s[n]                                                                 ## added line for checking payoff


In [25]:
@cuda.jit       # gpu test cases
def monte_carlo_andtheholygrail_gpu_faked(d_s, s_0, mu, sigma, T, Ki, Ko, temp_months, 
                                    pot,r,d_normals,  snowball_path, shiftp, blipp, regp, shiftdate, blipdate):
    

    # for shared memory (non)optimization
    shared = cuda.shared.array(shape=0, dtype=numba.float32)
    # load to shared memory
    path_offset = cuda.blockIdx.x * cuda.blockDim.x

    # ii - overall thread index
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x

    for n in range(ii, N_PATHS, stride):
        
        # print("blo got fucked\n")
        min = s_0
        price  =-999
            #cant instantiate lists in here ffs, need to pass it in instead
        snowball_path[0] = s_0

        earlyexit = False
        ki = False
        mald = False

        for t in range(T):
            # pre shared memory b_motion
            # b_motion = d_normals[n + t * N_PATHS]

            # post shared memory b_motion
            shared[cuda.threadIdx.x] = d_normals[path_offset + cuda.threadIdx.x + t * N_PATHS]

            dt = 1/T

            
            # FAKED PART HERE 211
            # if t >= 300 :
            #     snowball_path[t+1] = 11
            # elif t == 75:
            #     snowball_path[t+1] = 9
            # else: snowball_path[t+1] = 9
            
            # FAKED PART HERE 211
            if t >= shiftdate :
                snowball_path[t+1] = shiftp
            elif t == blipdate:
                snowball_path[t+1] = blipp
            else: snowball_path[t+1] = regp
        


            # pre shared memory b_motion
            # ds = snowball_path[t] * mu * dt + snowball_path[t] * sigma * b_motion * math.sqrt(dt) 
            # post shared memory b_motion
            # ds = snowball_path[t] * mu * dt + snowball_path[t] * sigma * shared[cuda.threadIdx.x] * math.sqrt(dt) 
            #         # no adjusting list sizes in cuda :(
            # # snowball_path.append(snowball_path[t]+ds)
            # snowball_path[t+1] = snowball_path[t] + ds
            

            # ki = snowball_path[t] + ds
            if snowball_path[t+1] <= Ki:
                ki = True
                if min > snowball_path[t+1]:
                    min = snowball_path[t+1]        # getting lowest drop

            # if t+1+startday in temp_months:     #startday used to fake a start date in code
            #     if snowball_path[t+1] >= Ko:
            #         price =  pot * t/365     # should turn t into int
            #         # return snowball_path, price
            #         d_s[n] =  price * math.exp(-r * t/T)   # accounting for r
            #         earlyexit = True
            #         break

            if not mald:
                for month in (0,1,2,3,4,5,6,7,8,9,10,11):                # need to do this instead because contains (in) and range are disabled
                    if t+1+startday == temp_months[month]:     #startday used to fake a start date in code
                        # price = t+1+startday
                        if snowball_path[t+1] >= Ko:
                            price =  pot * t/365     # should turn t into int
                            # return snowball_path, price
                            d_s[n] =  price * math.exp(-r * t/T)   # accounting for r
                            earlyexit = True
                            mald = True
                            # print("blo got fucked\n")
                            break
            # if t+1+startday == temp_months[0]:     #startday used to fake a start date in code
            #     price = t+1+startday
            #     if snowball_path[t+1] >= Ko:
            #         price =  pot * t/365     # should turn t into int
            #         # return snowball_path, price
            #         d_s[n] =  price * math.exp(-r * t/T)   # accounting for r
            #         earlyexit = True
            #         # print("blo got fucked\n")
            #         # break

        
        if not earlyexit:       # to prevent early exit getting out of bdds error
            # print("blo got fucked")
            # did not get knocked up or down
            price = pot
            # t  =T 
                        # CAN'T USE T CUZ CUDA IS FUCKING SHIT so use -1 instead
                        # or not ig T works now :sob:
            if ki and snowball_path[T] <= Ki:          # blo got knocked down and never recovered
                # price = min - s_0
                price = min - s_0
            elif ki and snowball_path[T] <= Ko:          # blo got knocked down for a bit but finished above Ki
                # print("blo got fucked")
                price =0
        # d_s[n] = price * math.exp(-r) * (-1) ** earlyexit 
            d_s[n] = price * math.exp(-r) 

        # d_s[n] = price


In [42]:
       # idk why tutorial uses cupy arrays but they dont fuckign work
# for normal run
# randoms_gpu = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
# for cpu run
randoms_gpu_but_cpu = cupy.asnumpy(randoms_gpu)
# for fake
randoms_gpu1 = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
randoms_gpu2 = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
randoms_gpu3 = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
randoms_gpu4 = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
randoms_gpu5 = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)

number_of_threads = 256
number_of_blocks = (N_PATHS-1) // number_of_threads + 1

# output = cupy.zeros(N_PATHS, dtype=cupy.float32)
output = cupy.zeros(N_PATHS, dtype=cupy.float32)

output0 = cupy.zeros(N_PATHS, dtype=cupy.float32)
output0 = cupy.asnumpy(output0)

output1 = cupy.zeros(N_PATHS, dtype=cupy.float32)
output2 = cupy.zeros(N_PATHS, dtype=cupy.float32)
output3 = cupy.zeros(N_PATHS, dtype=cupy.float32)
output4 = cupy.zeros(N_PATHS, dtype=cupy.float32)
output5 = cupy.zeros(N_PATHS, dtype=cupy.float32)

# shiftdate = 300
# blipdate = 75                    # is at cpu for now

# highp = 11
# medp = 9
# lowp = 7

# shared_buffer_size = number_of_threads * 4
# output =  np.zeros(N_PATHS, dtype=np.float32)
snowball_path = cupy.zeros(T+1, dtype=cupy.float32)               # need to do this beacue cupy does not support mem alloc
snowball_path_but_cpu = cupy.asnumpy(snowball_path)            # apparenlty usng only a single snowball path fucks stuff up??? shall now be deprecated.


snowball_path_holder = np.zeros(N_PATHS, dtype=(np.float32,T+1+1))# extra 1 is for storing payoff
snowball_path_holder = cupy.array(snowball_path_holder)
snowball_path_holder_but_cpu = np.zeros(N_PATHS, dtype=(np.float32,T+1+1))# extra 1 is for storing payoff
# snowball_path_holder_but_cpu.fill(np.zeros(T+1, dtype=np.float32))


# snowball_path_but_cpu = cupy.zeros(N_PATHS, dtype=cupy.float32)
# snowball_path_holder_but_cpu = cupy.asnumpy(output0)
# snowball_path_but_cpu.fill(cupy.asnumpy(snowball_path))
# snowball_path_but_cpu = cupy.asnumpy(snowball_path)



# snowball_path_holder_but_cpu = .fill(cupy.asnumpy(snowball_path))


# # for using CPU to run
# snowball_path = np.zeros(T+1, dtype=np.float32)
# randoms_gpu = cupy.asnumpy(randoms_gpu)
s = time.time()

# RUN THIS ONE UND#R NORMAL CIRCUMSTANCES
monte_carlo_andtheholygrail_gpu[(number_of_blocks,), (number_of_threads,)](output, np.float32(S0), np.float32(mu), 
                            np.float32(sigma), np.int32(T),np.float32(Ki),
                            np.float32(Ko), temp_months, np.float32(pot),
                             np.float32(r), randoms_gpu, snowball_path,  snowball_path_holder)
v = output.mean()
cuda.synchronize()
e = time.time()
print('time', e-s, 'v', v, 'avg time', (e-s)/N_PATHS)
# print (output)



# for CPU run
monte_carlo_andtheholygrail_gpu_but_cpu(output0, np.float32(S0), np.float32(mu), 
                            np.float32(sigma), np.int32(T),np.float32(Ki),
                            np.float32(Ko), temp_months, np.float32(pot),
                             np.float32(r), randoms_gpu_but_cpu, snowball_path_but_cpu, snowball_path_holder_but_cpu)

v = output0.mean()
# cuda.synchronize()
e = time.time()
print('time', e-s, 'v', v, 'avg time cpu', (e-s)/N_PATHS)
# print (output)


# USED FOR TESTING FAKE PATHS
# #                    premature exit
# monte_carlo_andtheholygrail_gpu_faked[(number_of_blocks,), (number_of_threads,)](output1, np.float32(S0), np.float32(mu), 
#                             np.float32(sigma), np.int32(T),np.float32(Ki),
#                             np.float32(Ko), temp_months, np.float32(pot),
#                              np.float32(r), randoms_gpu1, snowball_path, highp ,  medp,   medp, shiftdate, blipdate)
#                                                                       # shiftp,  blipp,  regp, 
# v = output1.mean()
# cuda.synchronize()
# e = time.time()
# print('v', v)
# #                    no exit
# monte_carlo_andtheholygrail_gpu_faked[(number_of_blocks,), (number_of_threads,)](output2, np.float32(S0), np.float32(mu), 
#                             np.float32(sigma), np.int32(T),np.float32(Ki),
#                             np.float32(Ko), temp_months, np.float32(pot),
#                              np.float32(r), randoms_gpu2, snowball_path, medp ,  medp,   medp, shiftdate, blipdate)
#                                                                       # shiftp,  blipp,  regp, 
# v = output2.mean()
# cuda.synchronize()
# e = time.time()
# print('v', v)
# #                    dip & recover
# monte_carlo_andtheholygrail_gpu_faked[(number_of_blocks,), (number_of_threads,)](output3, np.float32(S0), np.float32(mu), 
#                             np.float32(sigma), np.int32(T),np.float32(Ki),
#                             np.float32(Ko), temp_months, np.float32(pot),
#                              np.float32(r), randoms_gpu3, snowball_path, medp ,  lowp,   medp, shiftdate, blipdate)
#                                                                       # shiftp,  blipp,  regp, 
# v = output3.mean()
# cuda.synchronize()
# e = time.time()
# print('v', v)
# #                    dip & nah id lose
# monte_carlo_andtheholygrail_gpu_faked[(number_of_blocks,), (number_of_threads,)](output4, np.float32(S0), np.float32(mu), 
#                             np.float32(sigma), np.int32(T),np.float32(Ki),
#                             np.float32(Ko), temp_months, np.float32(pot),
#                              np.float32(r), randoms_gpu4, snowball_path, lowp ,  lowp,   medp, shiftdate, blipdate)#!!!! blip low or med
#                                                                       # shiftp,  blipp,  regp, 
# v = output4.mean()
# cuda.synchronize()
# e = time.time()
# print('v', v)
# #                    dip & nah id win
# monte_carlo_andtheholygrail_gpu_faked[(number_of_blocks,), (number_of_threads,)](output5, np.float32(S0), np.float32(mu), 
#                             np.float32(sigma), np.int32(T),np.float32(Ki),
#                             np.float32(Ko), temp_months, np.float32(pot),
#                              np.float32(r), randoms_gpu5, snowball_path, highp ,  lowp,   medp, shiftdate, blipdate)
#                                                                       # shiftp,  blipp,  regp, 
# v = output5.mean()
# cuda.synchronize()
# e = time.time()
# print('v', v)


# print(randoms_gpu)
print("\n", output, "\n")
print(output0, "\n")
print("GPU\n")
print(snowball_path_holder, "\n")
print("CPU\n")
print(snowball_path_holder_but_cpu)

np.save('snowball_path_holder2', snowball_path_holder) # save the array to a file named 'array_file.npy'

time 0.011852264404296875 v 2.4459221 avg time 2.370452880859375e-08
time 1.181063175201416 v 2.4459217 avg time cpu 2.3621263504028322e-06

 [ 3.1111178  6.413707   4.901201  ...  1.4221227 -3.3481214  1.4221227] 

[ 3.1111178  6.413707   4.901201  ...  1.4221227 -3.348122   1.4221227] 

GPU

[[10.         9.976178   9.739369  ...  8.784355   8.961286   3.1111178]
 [10.        10.026927  10.060698  ...  9.957478   9.992323   6.413707 ]
 [10.        10.370656  10.5086975 ... 15.973631  16.125055   4.901201 ]
 ...
 [10.         9.9560375  9.840625  ... 11.244363  11.078797   1.4221227]
 [10.        10.091348  10.046168  ...  6.5839825  6.5671206 -3.3481214]
 [10.        10.233092  10.412094  ... 13.560455  13.834585   1.4221227]] 

CPU

[[10.         9.976178   9.739369  ...  8.784357   8.961288   3.1111178]
 [10.        10.026927  10.060698  ...  9.957478   9.992323   6.413707 ]
 [10.        10.370656  10.5086975 ... 15.9736395 16.125063   4.901201 ]
 ...
 [10.         9.9560375  9.840

In [43]:
# A = np.load("snowball_path_holder_but_cpu_1.npy")
# B = np.load("snowball_path_holder_but_cpu_2.npy")

A = np.load("snowball_path_holder1.npy")
B = np.load("snowball_path_holder2.npy")
print(np.array_equal(A,B))

True


### I give up we doing CUPY GPU now
all my boys hate NUMBA GPU acceleration <br>
nvm im not coding in C :vomit: <br>
going back to Numba instead


In [16]:
randoms_gpu = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
randoms_cpu = np_randoms = cupy.asnumpy(randoms_gpu)

In [17]:
# @cuda.jit
@njit( fastmath = True, parallel=True) # parallel also does jack shit

def monte_carlo_andtheholygrail_gpu(d_s, s_0, mu, sigma, T, Ki, Ko, temp_months, 
                                    pot,r,d_normals):
    # shared = cuda.shared.array(shape=0, dtype=numba.float32)
    # # load to shared memory
    # path_offset = cuda.blockIdx.x * cuda.blockDim.x


    # ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    # stride = cuda.gridDim.x * cuda.blockDim.x
    
    for n in range( N_PATHS):
    # for n in range(ii, N_PATHS, stride):
        min = s_0
        snowball_path = [s_0]     # array of size 1
        earlyexit = False
        # ko = False            # never used
        ki = False
        for t in range(T):
            # path
            b_motion = d_normals[n + t * N_PATHS]
            dt = 1/T
            ds = snowball_path[t] * mu * dt + snowball_path[t] * sigma * b_motion * math.sqrt(dt)  
            snowball_path.append(snowball_path[t]+ds)
            # pricing and break if need be
            if snowball_path[t+1] <= Ki:
                ki = True
                if min > snowball_path[t+1]:
                    min = snowball_path[t+1]        # getting lowest drop
            
            if t+1+startday in temp_months:     #startday used to fake a start date in code
                if snowball_path[t+1] >= Ko:
                    price =  pot * t/365     # should turn t into int
                    # return snowball_path, price
                    d_s[n] =  price * math.exp(-r * t/T)   # accounting for r
                    earlyexit = True
                    break
        
        if not earlyexit:       # to prevent early exit getting out of bdds error
            # did not get knocked up or down
            price = pot
            if ki and snowball_path[T] <= Ki:          # blo got knocked down and never recovered
                price = min - s_0
            elif ki:          # blo got knocked down for a bit but finished above Ki
                price =0
            d_s[n] = price * math.exp(-r)